Aquest codi llegeix els fitxers amb les dades de les estadístiques dels jugadors i en treu les classificacions.

In [8]:
!pip install netCDF4


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: c:\users\admin\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd

c:\users\admin\appdata\local\programs\python\python39\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [14]:
dataarray = xr.open_dataset('stats.nc', engine='scipy')
dataarray

SystemError: initialization of _internal failed without raising an exception

SystemError: initialization of _internal failed without raising an exception